NBA player stats analysis
Q: Which stats are most correlated with salaries? 
Q: Can we forecast salaries to help determine players that may be over- and/or under-valued?

In [ ]:
#1: import core packages
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import time 
import os

In [ ]:
#1a: load existing data if available
if os.path.exists('nba_stats_with_salary_2025.csv'):
    print("Found existing data file! Loading...")
    df_merged = pd.read_csv('nba_stats_with_salary_2025.csv')
    df_merged_qualified = df_merged[df_merged['MP'] >= 15].copy()
    
    print(f"Loaded {len(df_merged)} players")
    print(f"Qualified players (15+ MPG): {len(df_merged_qualified)}")
    print("\nYou can now skip to Cell 20 to analyze the data!")
    print("Or continue from Cell 2 to re-scrape fresh data.")
else:
    print("No existing data found. Run cells 2+ to scrape data.")

In [ ]:
#2: set up the url 
url = 'https://www.basketball-reference.com/leagues/NBA_2025_per_game.html'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

In [ ]:
#3: request the data
print(f"Fetching data from {url}...")
response = requests.get(url, headers=headers)
print(f"Status code: {response.status_code}")


In [ ]:
#4: Parse the HTML
soup = BeautifulSoup(response.content, 'lxml')
table = soup.find('table', {'id': 'per_game_stats'})
print("Table Found!" if table else "Table not found")

In [ ]:
#5: extract column headers
headers_list = [ ]
for th in table.find('thead').find_all('th'):
    headers_list.append(th.text.strip())
print(f"Columns: {headers_list}")

In [ ]:
#6: extract all rows 
rows = []
for tr in table.find('tbody').find_all('tr'):
    # Skip header rows that appear in the middle
    if tr.find('th', {'scope': 'row'}) is None:
        continue
    
    row = []
    for td in tr.find_all(['th', 'td']):
        row.append(td.text.strip())
    
    if row:
        rows.append(row)

print(f"Scraped {len(rows)} player records")


In [ ]:
#7: create data frame
df = pd.DataFrame(rows, columns=headers_list)
print(df.head())


In [ ]:
#8: Convert numeric columns
numeric_cols = df.columns[5:]  # Stats start after Pos column
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print("\nData types after conversion:")
print(df.dtypes)

In [ ]:
# Cell 9: Explore the data
print(f"\nTotal players: {len(df)}")
print(f"\nBasic stats:\n{df[['Player', 'Team', 'PTS', 'TRB', 'AST']].describe()}")

In [ ]:
# Cell 10: Top 10 scorers
print("\nTop 10 Scorers (PPG):")
top_scorers = df.nlargest(10, 'PTS')[['Player', 'Team', 'PTS', 'TRB', 'AST']]
print(top_scorers)

In [ ]:
# Cell 11: Calculate simplified PER
def calculate_simple_per(row):
    """
    Calculate a simplified Player Efficiency Rating
    
    Formula components:
    - Points, Rebounds, Assists, Steals, Blocks (positive)
    - Missed FG, Missed FT, Turnovers (negative)
    - Adjusted per minute played
    """
    try:
        # Get stats (using .get() to handle missing values)
        pts = row['PTS']
        trb = row['TRB']
        ast = row['AST']
        stl = row['STL']
        blk = row['BLK']
        
        fga = row['FGA']
        fgm = row['FG']
        fta = row['FTA']
        ftm = row['FT']
        tov = row['TOV']
        
        mp = row['MP']
        
        # Calculate missed shots
        missed_fg = fga - fgm
        missed_ft = fta - ftm
        
        # Simplified PER formula
        per = (pts + trb + ast + stl + blk - missed_fg - missed_ft - tov) / mp if mp > 0 else 0
        
        # Scale it to look more like traditional PER (multiply by a factor)
        per = per * 10
        
        return round(per, 2)
    
    except:
        return None

In [ ]:
# Apply the function to each row
df['Simple_PER'] = df.apply(calculate_simple_per, axis=1)

print("Simple PER calculated!")
print(f"\nPER Stats:")
print(f"Average PER: {df['Simple_PER'].mean():.2f}")
print(f"Max PER: {df['Simple_PER'].max():.2f}")
print(f"Min PER: {df['Simple_PER'].min():.2f}")

In [ ]:
# Filter for players with meaningful minutes (at least 15 MPG)
df_qualified = df[df['MP'] >= 15].copy()

print("Simple PER calculated!")
print(f"\nAll players - PER Stats:")
print(f"Average PER: {df['Simple_PER'].mean():.2f}")
print(f"\nQualified players (15+ MPG) - PER Stats:")
print(f"Average PER: {df_qualified['Simple_PER'].mean():.2f}")
print(f"Total qualified players: {len(df_qualified)}")

In [ ]:
# Cell 12: Top 10 by PER (qualified players only)
print("\nTop 10 Players by Simple PER (min 15 MPG):")
top_per = df_qualified.nlargest(10, 'Simple_PER')[['Player', 'Team', 'MP', 'PTS', 'TRB', 'AST', 'Simple_PER']]
print(top_per)

In [ ]:
# Cell 13: Save to CSV with PER
df.to_csv('nba_player_stats_2025.csv', index=False)
print("\nData with Simple PER saved to nba_player_stats_2025.csv")

In [ ]:
# scrape player salaries for upcoming season
# Cell 14: Scrape player salaries
salary_url = 'https://www.basketball-reference.com/contracts/players.html'

print(f"Fetching salary data from {salary_url}...")
salary_response = requests.get(salary_url, headers=headers)
print(f"Status code: {salary_response.status_code}")

In [ ]:
# Cell 15: Parse salary table
salary_soup = BeautifulSoup(salary_response.content, 'lxml')
salary_table = salary_soup.find('table', {'id': 'player-contracts'})
print("Salary table found!" if salary_table else "Salary table not found")

In [ ]:
# Cell 16: Extract salary data with correct column names
correct_headers = ['Rk', 'Player', 'Tm', '2025-26', '2026-27', '2027-28', '2028-29', '2029-30', '2030-31', 'Guaranteed']

salary_rows = []
for tr in salary_table.find('tbody').find_all('tr'):
    row = []
    
    # Get all cells in the row
    cells = tr.find_all(['th', 'td'])
    
    for cell in cells:
        row.append(cell.text.strip())
    
    # Only keep rows that have 10 columns (matching our headers)
    if len(row) == 10:
        salary_rows.append(row)

print(f"Scraped {len(salary_rows)} salary records with 10 columns")
if salary_rows:
    print(f"Sample first row: {salary_rows[0]}")



In [ ]:
# Cell 17: Create salary DataFrame with correct columns
df_salary = pd.DataFrame(salary_rows, columns=correct_headers)
print(f"\nSalary columns: {df_salary.columns.tolist()}")
print(f"Shape: {df_salary.shape}")
print(df_salary[['Player', 'Tm', '2025-26']].head(10))

In [ ]:
# Cell 18: Clean salary data - just grab Player and 2025-26
df_salary_clean = df_salary[['Player', '2025-26']].copy()

# Clean the salary column
df_salary_clean['Salary_2025_26'] = df_salary_clean['2025-26'].str.replace('$', '').str.replace(',', '')
df_salary_clean['Salary_2025_26'] = pd.to_numeric(df_salary_clean['Salary_2025_26'], errors='coerce')

# Drop the raw column
df_salary_clean = df_salary_clean[['Player', 'Salary_2025_26']].copy()

print(f"Salary data cleaned. Sample:")
print(df_salary_clean.head(15))
print(f"\nTotal players with salary: {len(df_salary_clean)}")

In [ ]:
# Cell 19: Merge salary with stats (handling multiple team entries)

# For players with multiple team entries, we want to merge salary to ALL their rows
# The salary data has each player once, stats may have them multiple times

df_merged = df.merge(df_salary_clean, on='Player', how='left')

print(f"\nMerge complete!")
print(f"Total stat rows (including multi-team players): {len(df_merged)}")
print(f"Unique players: {df_merged['Player'].nunique()}")
print(f"Total rows with salary data: {df_merged['Salary_2025_26'].notna().sum()}")
print(f"Unique players with salary: {df_merged[df_merged['Salary_2025_26'].notna()]['Player'].nunique()}")

# Create qualified dataset - use TOT (total) rows for multi-team players when available
# If no TOT row, use their individual team rows
df_merged_qualified = df_merged[df_merged['MP'] >= 15].copy()

print(f"\nQualified players (15+ MPG):")
print(f"Total qualified rows: {len(df_merged_qualified)}")
print(f"Qualified rows with salary: {df_merged_qualified['Salary_2025_26'].notna().sum()}")

# Show examples of players without salary
missing_salary = df_merged_qualified[df_merged_qualified['Salary_2025_26'].isna()].sort_values('PTS', ascending=False)
print(f"\nTop 10 qualified players missing salary data:")
print(missing_salary[['Player', 'Team', 'MP', 'PTS']].head(10))

In [ ]:
# Cell 20: Top 10 players by PER with salary
# For players on multiple teams, prefer their TOT (total) row
df_for_ranking = df_merged_qualified.copy()

# Create a preference: TOT rows first, then single team rows
df_for_ranking['is_tot'] = df_for_ranking['Team'] == 'TOT'

# Remove duplicate players, keeping TOT if available
df_for_ranking = df_for_ranking.sort_values('is_tot', ascending=False).drop_duplicates(subset='Player', keep='first')

print(f"\nUnique qualified players for ranking: {len(df_for_ranking)}")
print("\nTop 10 Players by Simple PER with Salary (min 15 MPG):")
top_per_salary = df_for_ranking.nlargest(10, 'Simple_PER')[['Player', 'Team', 'MP', 'PTS', 'Simple_PER', 'Salary_2025_26']]
print(top_per_salary)

In [ ]:
# Cell 21: Calculate value with better filters
df_for_ranking['Value'] = df_for_ranking['Simple_PER'] / (df_for_ranking['Salary_2025_26'] / 1_000_000)
df_for_ranking['Value'] = df_for_ranking['Value'].round(2)

# Filter for players making at least $5M
high_earners = df_for_ranking[df_for_ranking['Salary_2025_26'] >= 5_000_000].copy()

print("\nTop 10 Best Value Players (PER per $1M, min $5M salary):")
top_value = high_earners.nlargest(10, 'Value')[['Player', 'Team', 'Simple_PER', 'Salary_2025_26', 'Value']]
print(top_value)

print("\n\nTop 10 Players by Simple PER (min 15 MPG):")
top_performers = df_for_ranking.nlargest(10, 'Simple_PER')[['Player', 'Team', 'MP', 'PTS', 'TRB', 'AST', 'Simple_PER', 'Salary_2025_26']]
print(top_performers)

print("\n\nMost Overpaid (min $20M salary, sorted by worst PER):")
overpaid = df_for_ranking[df_for_ranking['Salary_2025_26'] >= 20_000_000].nsmallest(10, 'Simple_PER')[['Player', 'Team', 'Simple_PER', 'Salary_2025_26', 'Value']]
print(overpaid)

In [ ]:
# Cell 22: Save merged data
df_merged.to_csv('nba_stats_with_salary_2025.csv', index=False)
print("\nFull data with salary saved to nba_stats_with_salary_2025.csv")